# 湖泊模型 (Lake Model)

`湖泊 (Lake)` 类模拟了一个具有恒定表面积的湖泊或水库。它通过计算入流、出流和蒸发来追踪水量和水位。本Jupyter Notebook概述了 `Lake` 模型并对其行为进行了仿真。

## 状态变量

湖泊的状态由以下变量定义：

- `volume` (float): 湖中当前的水量 (m³)。
- `water_level` (float): 水位，由水量和表面积计算得出 (m)。
- `outflow` (float): 湖泊的出流量 (m³/s)，通常由下游组件决定。

## 参数

湖泊的物理属性使用这些参数设置：

- `surface_area` (float): 湖泊的表面积 (m²)。
- `max_volume` (float): 湖泊的最大蓄水容量 (m³)。
- `evaporation_rate_m_per_s` (float): 湖泊表面的水分蒸发速率 (m/s)。

## 仿真示例

以下代码模拟了一个湖泊随时间的变化。它从一个初始水量开始，并施加一个恒定的入流。在仿真过程中，出流量会发生变化，以展示其对湖泊水量和水位的影响。然后将结果绘制成图表。

**注意:** 要运行此Notebook，请确保您已安装 `matplotlib` (`pip install matplotlib`)，并且您是从项目的根目录运行Jupyter服务器的。

In [ ]:
import matplotlib.pyplot as plt
from swp.simulation_identification.physical_objects.lake import Lake
from swp.core.interfaces import State, Parameters

# 湖泊的初始状态和参数
initial_state = State(volume=5e6, water_level=0.0, outflow=0.0)
parameters = Parameters(
    surface_area=1e6,      # 1 平方公里
    max_volume=10e6,       # 1000万立方米
    evaporation_rate_m_per_s=1e-8 # 约 0.864 毫米/天
)

# 创建一个Lake实例
lake = Lake(name="main_lake", initial_state=initial_state, parameters=parameters)

# 仿真设置
dt = 3600  # 时间步长（秒）(1小时)
simulation_duration = 3600 * 24 * 5  # 5天
num_steps = int(simulation_duration / dt)
inflow_rate = 10.0  # 恒定入流 (m³/s)

# 存储结果用于绘图
history = []

# 运行仿真循环
for t in range(num_steps):
    # 假设出流量由下游用户请求
    if t > num_steps // 2:
        requested_outflow = 15.0 # 增加的需求
    else:
        requested_outflow = 5.0 # 正常需求
        
    action = {'outflow': requested_outflow}
    
    lake.set_inflow(inflow_rate)
    current_state = lake.step(action=action, dt=dt)
    history.append(current_state.copy())

# 打印最终状态
print("最终状态:", lake.get_state())

# 准备绘图数据
time_in_hours = [i * dt / 3600 for i in range(num_steps)]
volumes = [s['volume'] for s in history]
water_levels = [s['water_level'] for s in history]
outflows = [s['outflow'] for s in history]

# 绘制结果
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(time_in_hours, [v / 1e6 for v in volumes], label='水量 (Volume)')
ax1.set_ylabel('水量 (百万 m³)')
ax1.set_title('湖泊仿真')
ax1.grid(True)
ax1.legend()

ax2.plot(time_in_hours, water_levels, label='水位 (Water Level)', color='orange')
ax2.set_xlabel('时间 (小时)')
ax2.set_ylabel('水位 (m)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()
